# Recommender Systems

In [1]:
from random import gauss as gs, uniform as uni, seed
import numpy as np
import pandas as pd

Recommender systems can be classified along various lines. One fundamental distinction is **content-based** vs. **collaborative-filtering** systems.

To illustrate this, consider two different strategies: (a) I recommend items to you that are similar to other items you've used/bought/read/watched; and (b) I recommend items to you that people similar to you have used/bought/read/watched. The first is the **content-based strategy**; the second is the **collaborative-filtering strategy**. 

Another distinction drawn is in whether (a) the system uses existing ratings to compute user-user or item-item similarity, or (b) the system uses machine learning techniques to make predictions. Recommenders of the first sort are called **memory-based**; recommenders of the second sort are called **model-based**.

## Content-Based Systems

The basic idea here is to recommend items to a user that are *similar* to items that the user has already enjoyed. Suppose we represent TV shows as rows, where the columns represent various features of these TV shows. These features might be things like the presence of a certain actor or the show fitting into a particular genre etc. We'll just use binary features here, perhaps the result of some one-hot encoding:

In [2]:
tv_shows = np.array([[0, 1, 1, 0, 1, 1, 1],
                   [0, 0, 0, 1, 1, 1, 0],
                   [1, 1, 1, 0, 0, 1, 1],
                   [0, 1, 1, 1, 0, 0, 1]])

tv_shows

array([[0, 1, 1, 0, 1, 1, 1],
       [0, 0, 0, 1, 1, 1, 0],
       [1, 1, 1, 0, 0, 1, 1],
       [0, 1, 1, 1, 0, 0, 1]])

Bob likes the TV Show represented by Row \#1. Which show (row) should we recommend to Bob? We can calculate the cosine of the angle $\theta$ between two vectors $\vec{a}$ and $\vec{b}$ as follows: $\cos(\theta) = \frac{\vec{a}\cdot\vec{b}}{|\vec{a}||\vec{b}|}$

In [3]:
numerators = np.array(
    [tv_shows[0].dot(tv_shows[i]) for i in range(1, 4)]
)
denominators = np.array(
    [np.sqrt(sum(tv_shows[0]**2)) * np.sqrt(sum(tv_shows[i]**2)) for i in range(1, 4)])

numerators / denominators

array([0.51639778, 0.8       , 0.67082039])

Since the cosine similarity to Row \#1 is highest for Row \#3, we would recommend this film.

***

## Collaborative Filtering

Now the idea is to recommend items to a user based on what *similar* users have enjoyed. Suppose we have the following recording of explicit ratings of five items by three users:

In [4]:
users = np.array([[5, 4, 3, 4, 5], [3, 1, 1, 2, 5], [4, 2, 3, 1, 4]])

new_user = np.array([5, 0, 0, 0, 0])

To which user is new_user most similar?

One metric is cosine similarity:

In [5]:
new_user_mag = 5

numerators = np.array([new_user.dot(users[i]) for i in range(3)])
denominators = np.array([np.sqrt(sum(new_user**2)) * np.sqrt(sum(users[i]**2)) for i in range(3)])

numerators / denominators

array([0.52414242, 0.47434165, 0.58976782])

But we could also use another metric, such as Pearson Correlation:

In [6]:
[np.corrcoef(new_user, users[i])[0, 1] for i in range(3)]

[0.5345224838248488, 0.20044593143431824, 0.5144957554275266]

For more on content-based vs. collaborative systems, see [this Wikipedia article](https://en.wikipedia.org/wiki/Collaborative_filtering) and [this blog post](https://towardsdatascience.com/recommendation-systems-models-and-evaluation-84944a84fb8e). [This post](https://dataconomy.com/2015/03/an-introduction-to-recommendation-engines/) on dataconomy is also useful.

***

## Matrix Factorization

Suppose we start with a matrix $R$ of users and products, where each cell records the ranking the relevant user gave to the relevant product. Very often we'll be able to record this data as a sparse matrix, because many users will not have ranked many items.

Imagine factoring this matrix into a user matrix and an item matrix: $R = PQ^T$. Then we could predict a user's ranking of a particular item simply by calculating $p^Tq$, the dot-product of a row of $P$ and a column of $Q^T$. Why? The user vector records the user's preferences with respect to certain latent features, while the item vector records how the item ranks with respect to those same latent features.

If we could effect such a factorization, then we could calculate *all* predictions, i.e. fill in the gaps in $R$, by solving for P and Q.

The isolation of these latent features can be achieved in various ways. But this is at heart a matter of **dimensionality reduction**, and so one way is with the [SVD](https://hackernoon.com/introduction-to-recommender-system-part-1-collaborative-filtering-singular-value-decomposition-44c9659c5e75).

An alternative is to use the method of Alternating Least Squares.

### Alternating Least-Squares (ALS)

ALS recommendation systems are often implemented in Spark architectures because of the appropriateness for distributed computing. ALS systems often involve very large datasets (consider how much data the recommendation engine for NETFLIX must have, for example!), and it is often useful to store them as sparse matrices, which Spark's ML library can handle. In fact, Spark's mllib even includes a "Rating" datatype! ALS is **collaborative** and **model-based**.

We're looking for two matrices (a user matrix and an item matrix) into which we can factor our ratings matrix. We can't of course solve for two matrices at once. But here's what we can do:

Make guesses of the values for P and Q. Then hold the values of one *constant* so that we can optimize for the values of the other!

Basically this converts our problem into a familiar *least-squares* problem. See [this page](https://textbooks.math.gatech.edu/ila/least-squares.html) and [this page](https://datasciencemadesimpler.wordpress.com/tag/alternating-least-squares/) for more details, but here's the basic idea:

If we have an equation $Ax = b$ for *non-square* $A$, then we have:

$A^TAx = A^Tb$ <br/>
Thus: <br/>
$x = (A^TA)^{-1}A^Tb$

This $(A^TA)^{-1}A^T$ **is the pseudo-inverse of** $A$.

In [7]:
np.random.seed(42)

A = np.random.rand(5, 5)
b = np.random.rand(5, 1)

In [8]:
np.linalg.inv(A.T.dot(A)).dot(A.T).dot(b)

array([[-226.17809008],
       [ 218.48756762],
       [-362.56506574],
       [ 147.90541511],
       [ 350.14590733]])

In [9]:
np.linalg.pinv(A).dot(b)

array([[-226.17808981],
       [ 218.48756735],
       [-362.56506531],
       [ 147.90541493],
       [ 350.14590691]])

"When we talk about collaborative filtering for recommender systems we want to solve the problem of our original matrix having millions of different dimensions, but our 'tastes' not being nearly as complex. Even if i’ve \[sic\] viewed hundreds of items they might just express a couple of different tastes. Here we can actually use matrix factorization to mathematically reduce the dimensionality of our original 'all users by all items' matrix into something much smaller that represents 'all items by some taste dimensions' and 'all users by some taste dimensions'. These dimensions are called ***latent or hidden features*** and we learn them from our data" ([Medium article: "ALS Implicit Collaborative Filtering"](https://medium.com/radon-dev/als-implicit-collaborative-filtering-5ed653ba39fe)).

#### Simple Example

Suppose Brian and Erin have rated five films:

In [10]:
ratings_arr = pd.DataFrame([[0, 1, 0, 0, 4], [0, 0, 0, 5, 0]], index=['brian', 'erin'],
             columns=['film' + str(i) for i in range(1, 6)])
ratings_arr

,film1,film2,film3,film4,film5
brian,0,1,0,0,4
erin,0,0,0,5,0


In [25]:
seed(100)
users = []

for _ in range(2):
    user = []
    for _ in range(10):
        user.append(gs(0, 1))
    users.append(user)
users_arr = np.array(users)
users_arr

array([[ 0.67155333,  0.87331967,  0.20361655, -1.55034921, -0.12059128,
        -1.05927574,  0.38143697, -1.17342904,  0.96637182,  0.53248343],
       [ 2.22041991,  0.68901284,  0.85436449, -0.29504752, -0.62335055,
         1.5917369 ,  0.17920475,  0.60086339,  0.3474319 ,  0.85537186]])

In [14]:
seed(100)
items = []

for _ in range(5):
    item = []
    for _ in range(10):
        item.append(gs(0, 1))
    items.append(item)
items_arr = np.array(items)
items_arr

array([[ 0.67155333,  0.87331967,  0.20361655, -1.55034921, -0.12059128,
        -1.05927574,  0.38143697, -1.17342904,  0.96637182,  0.53248343],
       [ 2.22041991,  0.68901284,  0.85436449, -0.29504752, -0.62335055,
         1.5917369 ,  0.17920475,  0.60086339,  0.3474319 ,  0.85537186],
       [-1.80291827, -1.83311295,  0.60911087,  2.4250145 , -2.02233576,
        -0.73382756,  0.24510831, -0.53817586, -0.30637608, -0.41367264],
       [ 1.0027436 ,  0.03558136,  0.19013362, -1.27827915,  0.67648704,
         1.79506722,  0.63054322, -0.37947302, -1.35033057,  0.45576721],
       [ 0.42542416, -0.29962041, -2.48035968, -0.87457154, -1.23050164,
        -1.00629648,  0.1857537 , -1.174924  , -0.33108494,  1.29437514]])

In [15]:
users_arr.shape

(2, 10)

In [16]:
items_arr.shape

(5, 10)

In [17]:
users_arr.dot(items_arr.T)

array([[ 7.53516384,  1.26783511, -5.21738432,  0.36547776,  3.90803828],
       [ 1.26783511, 10.38970834, -6.10853643,  5.03189793, -1.63817674]])

In [18]:
#get new users matrix

brian_pref = np.linalg.pinv(items_arr).dot(ratings_arr.loc['brian', :])
print(brian_pref)
erin_pref = np.linalg.pinv(items_arr).dot(ratings_arr.loc['erin', :])
print(erin_pref)

[ 0.47099573 -0.11214938 -0.98527859  0.09266666 -0.64747458  0.00854642
 -0.11601818  0.09586291 -0.08579953  0.55693307]
[-0.23238655 -0.55138144  0.5415508  -0.71191248  0.27767197  0.81157767
  0.78557579 -1.03623954 -1.25490079  0.02606555]


In [19]:
items_arr.T.shape

(10, 5)

In [20]:
#get new ratings matrix

newbrian = brian_pref.dot(items_arr.T)
newbrian

array([-1.33226763e-15,  1.00000000e+00, -6.10622664e-16,  1.22124533e-15,
        4.00000000e+00])

In [21]:
newerin = erin_pref.dot(items_arr.T)
newerin

array([ 8.88178420e-16,  0.00000000e+00, -2.22044605e-16,  5.00000000e+00,
       -1.88737914e-15])

In [22]:
guess = np.vstack([newbrian, newerin])

err = 0
for i in range(2):
    for j in range(len(ratings_arr.values[i, :])):
        if ratings_arr.values[i, j] != 0:
            err += (ratings_arr.values[i, j] - guess[i, j])**2
print(err)

8.283039504820624e-30


#### Second Example

In [147]:
# Users: m x n (m users)
# Items: r x n (r items)
# Ratings: m x r

In [26]:
# If P = users and Q = items, then we want to approximate R = PQ^T
# Let's generate R.

seed(42)
ratings2 = []
for _ in range(100):
    user = []
    for _ in range(100):
        chance = gs(0, 0.4)
        if chance > 0.5:
            user.append(int(uni(1, 6)))
        else:
            user.append(0)
        if user.count(0) == 10:
            user[int(uni(0, 10))] = int(uni(1, 6))
    ratings2.append(user)
ratings_arr2 = np.array(ratings2)

In [27]:
#start with random guess for users matrix

users2 = []

for _ in range(100):
    user = []
    for _ in range(10):
        user.append(gs(0, 1))
    users2.append(user)
users_arr2 = np.array(users2)

In [28]:
#start with random guess for items matrix

items2 = []

for _ in range(100):
    item = []
    for _ in range(10):
        item.append(gs(0, 1))
    items2.append(item)
items_arr2 = np.array(items2)

In [29]:
guess = users_arr2.dot(items_arr2.T)

In [30]:
guess.shape == ratings_arr2.shape

True

In [31]:
ratings_arr2 - guess

array([[ 1.20022042, -4.79370844,  0.16745819, ...,  0.71515764,
         5.29265957,  2.34612024],
       [12.81954999,  1.09244517,  2.76883236, ...,  3.84048632,
         3.63986143,  1.251285  ],
       [-0.11924386,  0.1143655 , -0.52294795, ...,  3.37782545,
        -2.46092459,  0.03240239],
       ...,
       [ 1.78409526, -0.16181232,  1.29355933, ...,  6.03771907,
         1.97382977,  4.89026489],
       [-1.85314949,  4.16815931,  3.84911175, ..., -0.0342376 ,
        -5.73579374, -1.53629352],
       [ 2.51939362,  5.31266136, -1.7857387 , ...,  5.18275941,
        -2.55671716,  4.90783561]])

In [32]:
# error for random guess

err = (ratings_arr2 - guess)**2

np.sum(err)

122333.98763617325

In [46]:
def als(ratings, users, items, reps=10):
    
    '''
    iteratively:
    - find a new users matrix based on the items and ratings matrices
    - find a new items matrix based on the ratings and new users matrices
    - print the error of the new users*items matrix against the ratings matrix
    
    inputs: ratings, users, items matrices
            reps=10, number of reps to iterate through
            
    outputs: new guess for ratings matrix after reps
    
    '''
    
    ratings_cols = ratings.T
    for _ in range(reps):
        new_users = []
        for i in range(len(ratings)):
            #user = np.inv(items[i].reshape(-1, 1).dot(items[i].reshape(1, -1))).dot(ratings[i])
            user = np.linalg.pinv(items).dot(ratings[i])
            new_users.append(user)
        new_users = np.asarray(new_users)
        
        new_items = []
        for i in range(len(ratings)):
            item = np.linalg.pinv(new_users).dot(ratings_cols[i])
            new_items.append(item)
        new_items = np.asarray(new_items)
        
        guess = new_users.dot(new_items.T)
        err = 0
        for i in range(len(ratings)):
            for j in range(len(ratings[i])):
                if ratings[i, j] != 0:
                    err += (ratings[i, j] - guess[i, j])**2
        print(err)
        
        items = new_items
        
    return new_users.dot(new_items.T)

In [50]:
#what's the error for the new guess of our ratings matrix?

new_guess = als(ratings_arr2, users_arr2, items_arr2, reps = 10)
err = (ratings_arr2 - new_guess)**2
np.sum(err)

8679.95926804698
6511.270608291893
6272.528821101907
6190.959718973226
6153.148460896028
6133.158269096345
6121.902204676269
6115.079057339577
6110.53807458497
6107.255266508423


8433.122935300897